# Biology 351 Lab: Week 7

## Processing sequencing reads and mapping to a reference genome

Remember these pro-tips:
- triple-click on a line to highlight the entire line to copy
- set up a split screen to put this notebook next to your terminal window

## 0. Logging on to spydur:

First let's log into `spydur`. Update the code below and use it to log in. If your key setup worked last week, you should not need to enter a password, but if you are prompted, enter your netid password (it won't show up as typing anything)

In [ ]:
ssh yournetid@spydur

## 1. Setting up folders
Now let's make new folders to work in for this week's lab. After last week, you should have a personal folder in the `/shared_perickso/` directory on Spydur. Navigate there now.

Use `mkdir` to make a subfolder below their first name called Lab7. 

In the end you should have a folder at `/home/netid/shared_perickso/firstname/Lab7` and be in that folder (replacing your netid and first name).

## 2. Introducting variables in Unix

Just like we can save different object as variable names in R (think back to how we had two separate data tables named "mydata" and "mydata.melt" in week 2), we can also save text, numbers, and files to variable names in UNIX. The advantage of doing this is that it can save you a lot of typing, and in some cases make complex operations much simpler. The formula is `variable_name=variable_value`. We can then refer back to those variables by typing `${variable_name}`. Add your name after the `name=` and then run both lines of code to see what happens

In [ ]:
name=
echo ${name} "loves biology!"

## 3. Make a variable to save our working directory

Now we'll save the name of the Lab7 folder as our "working directory". The working directory is where you have files and run code for the current analysis. We will save the directory as a variable name.  That way, we can just type `${WD}` instead of the full directory for everything we want to do. Update the code below with with the path to your Lab7 folder and run it. It should print back the location of `${WD}`. Anytime you want to use a path to your current working directory, you can type `${WD}` insteads of `/home/netid/shared_perickso/firstname/Lab7`. Trust me, you'll thank me later. Enter your WD below, then use `echo ${WD}` to confirm that it saved correctly

In [ ]:
WD=
echo ${WD}

## 4. Set up an interactive job
My lab (the Erickson lab) "owns" one computer that is part of the network of computers that makes up the **Spydur** computing cluster. We will be running our jobs on that computer.  To ensure that everyone's jobs run with equal computing resources, we will tell the computer how many cores of computing power and how much memory we need. Copy the code below and run it in terminal so that all your future commands run on my branch of `spydur`. This will give us each 20GB of memory on 3 computing cores to run our current operations for the next 3 hours. 

In [ ]:
srun --pty -t 3:00:00 --mem=20G --partition erickson --ntasks-per-node=3 bash

## 5. Finding the right files
We left off last week after looking at FastQ and FastA files.

**Question: What is the difference between the two file types?  What information is stored in each?**


This week, we are going to take some of our sequencing reads from fastq files and learn how to process them and map them to a reference genome in a fasta file.

Use the appropriate Unix command from Table 2 to look at the file in `shared_perickso` named `samples.txt`. This file contains all of the information about all of the _Z. indianus_ pooled sequencing libraries my lab has sequenced, sorted by the collection date. Here is the metadata for that file: 
- Pool: The name of the sequencing library. 
- Location: The location flies were collected:
    - CM = Carter Mountain, Charlottesville, VA
    - FL = Fruit and Spice Park, Miami, FL
    - HPO = Hanover Peach Orchard, Richmond, VA
    - HCGA = Hillcrest Orchard, Ellijay, GA
    - CK = Cross Keys Orchard, Harrisonburg, VA
    - MA = Acton, MA (flies combined from multiple orchards)
    - CT = Lyman Orchards, Middleton, CT
    - PA = Linvilla Orchards, Media, PA
    - CH = Chiles Orchards, Crozet, VA
- Year: The year of sampling
- Sample Date: The day flies were collected
- females: The number of female flies in the pool
- males: The number of male flies in the pool
- total: The total number of flies in the pool

**Question: What are the names of the two samples that you and your partner will be comparing? The name should be "ZP_" followed by a number**


**Question: How many flies were squished up and pooled together for your samples?**


**(Challenge!) Question: Will the number of flies affect how much sequencing data you have? Why or why not?**

Decide now with your partner who is going to process which sample.
 
**Question: which of your two samples will you be working with going forward?**


## 6. Looking at the real fastq files

Our fastq files have a lot of data in them. The raw data are stored in a zipped, compressed form, indicated by a .gz at the end of the file.

Change directories to `/shared_perickso/fastq`. Use `ls -lh`  (those are both lower-case Ls) to see the sizes of all the files in bytes. G=gigabytes, M=megabytes, K=kilobytes. 

**Question: How big are the raw data files for your samples?**

One of these files might be bigger than your monthly cell phone data usage (or at least mine, I don't use my phone very much), which means they can take a long time to process. I ran a program that randomly downsampled the fastq files so that only 0.5% of sequencing the reads were retained. It was run equally on both the read_1 and read_2 files so that the same pairs of reads were retained in each file. So, we have 0.5% of the original paired-end reads from the original sample. These reads are saved in `/shared_perickso/dfastq`. Make sure you use this folder or your next steps will take forever!

**Question: Use a command from Table 2 to determine how many lines are in the "downsampled" files for your sample (make sure to run it on both the read_1 and read_2 files) and report those numbers here** 

**Question: How many sequencing reads does that represent (remember from last week how reads are represented in a fastq file!)?**

Using these downsampled files, we will be able to run examples of some of the key bioinformatics steps, without having to wait for a long time for each program to run.

## 7. Merging reads

Recall that we are using 150 bp, paired-end reads.

**Question: Why might paired end reads occasionally have overlapping sequence? What would have to be true for this to happen?**


Because some reads might overlap, the sequence in the overlapping part represents duplicated data that is not a true "replicate" of the sequencing information. Therefore, we wouldn't want to count both of the overlapping reads as separate counts when we calculate our allele frequencies. We need to get rid of this duplication by merging the reads together. Essentially, the computer finds the overlap, if it exists, and comibines the two shorter overlapping reads into one long read.

The software that we are going to use to merge our reads is called `bbmap`. The specific program we are using is called `merge.sh`. The program takes two input files, the `read_1` and `read_2` fastq files. It finds occasions where the reads overlap, and merges them into a single read.

It has two input files that it will use:
 - `in1`: The downsampled read_1 reads (from the dfastq folder, in fastq format)
 - `in2`: The downsampled read_2 reads (from the dfastq folder, in fastq format)
 
It also has three output files that it will generate:
 - `out`: The merged reads (combining overlapping read 1 and read 2 reads into a single read)
 - `outu1`: The unmerged read 1 reads (ie the read 1 reads that didn't overlap with their paired end) 
 - `outu2`: The unmerged read 2 reads (ie the read 2 reads that didn't overlap with their paired end)
 
Set up the code below to run `bbmerge.sh` on one of your two downsampled files. Just like careful pipetting is critical to the success of a wet lab, proper file naming is going to be critical in this lab! We will assign each file name as a variable. The final line of code will execute the program with the proper files. Your output files should be saved in `${WD}` (your Lab7 folder) and named as follows. Samplename refers to the ZP_# name of your pool.
 
How to name your files to save them in your Lab7 folder: (replace `samplename` with your ZP_ sample number)
 - `out`: `${WD}/samplename.downsampled.merged.fq`
 - `out1`: `${WD}/samplename_1.downsampled.unmerged.fq`
 - `out2`: `${WD}/samplename_2.downsampled.unmerged.fq`
 
To simplify the line of code that actually runs the program we will save each of these names as a variable, then run the program just referencing the variables. When you enter the code below, there cannot be any spaces between the variable name, the =, and the file name!

Set up the names of all your files in the code block below. Don't forget you can use `${WD}` as a shortcut! Once you have all the file names entered, run the five lines of code to save your input and output file names. 


In [ ]:
in1=
in2= 
out=
out1=
out2=


Copy and paste all 5 lines above into terminal to save all the file names 

Then run the code below to do the merging. Note that we are using the variable names that we set up above to tell the computer which files to use.

In [ ]:
bbmerge.sh in1=${in1} in2=${in2} out=${out} outu1=${out1} outu2=${out2} strict=t


Take a look at the output that prints from the bbmerge program. It reports a bunch of statistics about the merging process. Answer the questions below:

**Question: What % of your reads were merged (or joined) together?**


**Question: What do you think it means by "insert size"?**


**Question: The total number of merged and unmerged reads should add up to our original number of reads (which you calculated in step 6. Use the appropriate command from Table 2 to count the number of lines in your merged file and your unmerged read 1 file. Report all the numbers here. Do the numbers make sense?**


## 8. Mapping the reads

Now it's time to map or align your fastq files to the reference genome. The program we will use is called `bwa`, which stands for Burrows-Wheeler aligner; named after the two people who invented the algorithm that it uses. Recall that when we mapped reads by hand, you scanned the DNA sequence of the reference genome looking for similarities between it and your sequencing read. This is essentially what the `bwa` program does, but it has to scan all ~150,000,000 bases of our reference genome. 

Each person will run BWA twice: once on their merged reads, and once on their unmerged (still paired-end reads). 


**Question: how do you think the mapping algorithm works differently when it is using single reads (the merged reads) vs paired reads (the read_1 and read_2 files)? Remember the physical relationship between paired end reads.**

 The overall syntax of how we run the program is

`bwa mem -M -t 3 reference_genome fastq_file (fastq_file_2) > output_file.sam`

 - `mem` is the specific algorithm the computer uses to map the reads to the reference genome
 - `-M` is an option that makes the output files compatible with a program we will use in the future
 - `-t 3` tells the computer to use 3 computing cores (the number you have reserved from the previous step
 - `reference_genome` is the fasta file that contains our reference genome file (we looked at it last week)
 - `fastq_file` is the reads that you want to map
 - `fastq_file_2` is the optoinal paired end data for what you want to map
 - `>` means "save the output to the file name after this symbol". This is a very common way to save files in Linux
 - `output_file` is the name of the file we will save the mapping data to. It has to end in ".sam". Ideally it will have the same name as the input file but end in .sam instead of .fastq
 
 
Below you will need to set up the code to run the merged and paired end files through bwa. The lines that have a # in front of them are notes and will not be read by Linux. The lines without a # are the actual code. The second step is going to take 10-15 minutes to run most likely, so you can move on and start working on the next section once your code is running, then come back to answer the questions below about the output of bwa.

In [ ]:
%%bash

#set up the code below to map the merged reads

#path to the file with the reference genome (the fasta file we looked at last week in /shared_perickso)
ref=

# input merged read file that you created above (the full path to the merged output of your previous code)
merged_reads=

#file you want to save to. should be named: ZP_#_merged.sam (replace the # with your sample #)
merged_output=

#below is the code that will actually run the mapping for teh merged reads--this will likely be fast
bwa mem -M -t 3 $ref $merged_reads > $merged_output

#now set up the code below to map the unmerged reads--the ones that are still separate as read 1 and read 2

#full path to your unmerged read_1 fastq file (can copy from above)
read_1=

#full path  and name of your unmerged read_2 fastq file (can copy from above)
read_2=

#file that you want to save to. shouldbe named as above but with "unmerged" in the name
unmerged_output=

#use this code to run the mapping for the unmerged reads--you can expect this to take 10-15 minutes. 
bwa mem -M -t 3 $ref $read_1 $read_2 > $unmerged_output


Look at the output that printed to your screen after running each job:

**Question: how long did it take to run the two jobs?** 

**Question: these reads where downsampled to 1% of their original read depth; how long would it have taken to run the whole file this way?**


**Question: look at your output .sam files (hint: use `less -S filename` to look at them in an orderly way that uses tabs to organize the columns). What key information does the sam file appear to contain?**

  
**Question: Now read about the sam file format here: https://en.wikipedia.org/wiki/SAM_(file_format). What column would be important if you wanted to identify reads that were a poor match to the reference genome?**



## 9. Converting to a bam file
Now, we will convert our sam files into a binary format file called a bam file. The information remains the same, but it is compressed and easier for the computer to work with. However, once it is in binary format, it is no longer human-readable, so it will just look like random caharters if you tried to look at it with the `less` command.

The program that can  convert files to bam is called `samtools`, and we will use a tool within the package called  `view`. The option `-b` tells the computer to convert to bam. 

This is also the step where we can remove poorly mapped reads by setting a quality filter. The `-q 40` option below means that we will remove anything with a mapping quality lower than 40. This is the first of many important filtering steps that will happen!  You will need to repeat this step for both your merged and unmerged sam files. 


In [ ]:
#provide the output named of your merged bam file. Same as the .sam name, but replace .sam with .bam
merged_bam_name=

samtools view -b -q 40 --threads 3  $merged_output > $merged_bam_name

#now provide an output name for the unmerged bam
unmerged_bam_name=

samtools view -b -q 40 --threads 3  $unmerged_output > $unmerged_bam_name


**Question: Take a look at one of the bam files you just made. What does it look like inside?**


Now that we have the files in .bam format, they are easier for the computer to work with. Next we need to sort them. Sorting puts all the mapped reads in order along the reference genome. Sorting is required for many downsteram steps. The tool we will use is called `sort` and the program syntax is simple: 
`samtools sort input_file > output_file`. You'll use this below to generate the command using your file names below.

You should include the word "sort" in your sorted file name.

In [ ]:

#now write the command using the example above. You'll need to consider what should be the input oand output. 
#your input file is the same as the merged bam name you created above! 




#now write the command to make your other sorted file.



## 10. Combine files
Now that our bam files are created and sorted, we can combine them into a final bam file containing all of our mapped reads (from the merged and unmerged). The syntax for this command is different; instead of using a > to tell the program where to save the file, the name of the **output** file is the first argument:

`samtools merge output_file input_file_1 input_file_2`

**Your output file should be named ZP_#_final.bam and saved in your Lab7 folder--I'll be checking to make sure this file is here!** 


In [ ]:
#use the syntax above to write your command below, then run it


Ta da! We are done for today. We went from raw data to a sorted, combined file of mapped reads. Next week, we will use this file to identify the locations of SNPs in the genome. 

When you are done, hit file> print preview. Then print the resulting window to a PDF and upload to Blackboard for Lab 7. 